In [ ]:
# Load all the necessary modules

import os
import torch
import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import librosa

In [5]:
# Load metadata CSV file
path = "/itf-fi-ml/shared/users/annammc/gestures/audio"
audiofile_list = glob.glob(os.path.join(path, "*.wav"))
# videofile_list = glob.glob(os.path.join(path, "*.mov"))
# midifile_list = glob.glob(os.path.join(path, "*.mid"))
# mocapcsv_list = glob.glob(os.path.join(path, "*.csv"))


In [11]:
for audio_file_path in audiofile_list:
    
    y, sr = librosa.load(audio_file_path, sr=None)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    chromagram = librosa.feature.chroma_stft(y=y, sr=sr)
    tempogram = librosa.feature.tempogram(y=y, sr=sr)

    basename = os.path.splitext(os.path.basename(audio_file_path))[0] 

    # Stack the features into a 2D array
    audio_feature = np.vstack((mfcc, chromagram, tempogram))

    # Save the feature array into a .npy file
    np.save(f"/itf-fi-ml/shared/users/annammc/Anna/audio/{basename}.npy", audio_feature)

In [12]:
db_gesture = "/itf-fi-ml/shared/users/annammc/Anna/audio/*"
files = glob.glob(db_gesture)
filename = [os.path.basename(item) for item in files]

target = [item.split("_")[-3] for item in files]

label_encoder = LabelEncoder()
target_idx = label_encoder.fit_transform(target) # Change class names to numeric

dict = {'audio_npy': filename, 'target': target, 'target_idx': target_idx}
dataset_all = pd.DataFrame(dict)

dataset_all.to_csv('metadata.csv')


In [14]:
X = dataset_all['audio_npy']
y = dataset_all['target_idx']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

In [ ]:
train_df = pd.DataFrame({'filename': list(X_train), 'target': list(y_train)})
test_df = pd.DataFrame({'filename':list(X_test), 'target': list(y_test)})

train_df.to_csv("/itf-fi-ml/shared/users/annammc/Anna/audio/train.csv") 
test_df.to_csv("/itf-fi-ml/shared/users/annammc/Anna/audio/test.csv")